# Interactive Functions 

In [1]:
import pandas as pd
from siuba import *

import numpy as np
from plotnine import *

import ipywidgets as widgets
from ipywidgets import *
from IPython.display import Markdown
from IPython.core.display import display

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = "{:.2f}".format

In [3]:
df = pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/function_data.csv', low_memory=False).drop('Unnamed: 0', axis=1)

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [4]:
df.sample(5)

,prefix,project_no,agency,prepared_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,ftip_no,project_location,type_of_work,seq,mpo,prepared_y
7344,HPLULN,6071(041),Orange County Transportation Authority,2015-12-08,74921.00,-68000.00,0.00,Authorized,6071,12,ORA100507,Beach Boulevard (sr 39) At I-405 Interchange,Remove Offramp & Const. 4th Northbound Lane,2,SCAG,2015-01-01
7702,CML,5109(186),Bakersfield,2015-12-08,-16545.06,0.00,664.16,Authorized,5109,6,KER120506,Intersection Of Stine Road & Woodmere Drive,Traffic Signals & Lighting Systems,2,KCOG,2015-01-01
15796,STP,5008(159),Stockton,2019-12-13,-339558.65,0.00,-750280.08,Authorized,5008,10,SJ07-2026,"I-5/french Camp Interchange (in Stockton, From...","Landscape, Install Landscaping At The Interchange",4,SJCG,2019-01-01
4402,DEMO06,5934(166),San Francisco County,2017-05-11,-163512.93,0.00,-138864.31,Authorized,5934,4,SF-070025,19th Avenue From Lincoln Way To Wawona,Landscape,3,MTC,2017-01-01
4847,STP,5241(048),Ceres,2017-02-28,-93313.67,0.00,-116899.99,Authorized,5241,10,VA02,Mitchell Road (phase Iii) - Whitmore Avenue To...,Overlay,5,STANCOG,2017-01-01


## Full Function for Prefix

In [5]:
    @interact
    
    def dla_get_prefix(place=df.agency.sort_values().unique().tolist()):
        
        agencies = df[df.agency==place]
    
        prefix_count_n = agencies >> count(_.prefix)
    
        display(Markdown(f"**Summary Statistics for {place}**"))
        display(Markdown(f"The number of obligations {place} has is {len(agencies)}"))
        
        display(Markdown(f"The number of prefix codes {place} uses is {len(prefix_count_n)}"))
        
        
        
        # for the table- using one as some agencies only have one entry
        pd.set_option("display.max_columns", None)
        display(df[df.agency == place][['fed_requested','ac_requested','total_requested']].describe())
        
        display(Markdown(f"**Top Project Types in {place}**"))
        display((df[df.agency == place] >> count(_.type_of_work) >> arrange(-_.n)).head(5)) 
        # graphs 
    
         

    
        ax1 = (prefix_count_n
            >> ggplot(aes("prefix", "n", fill="prefix")) 
               + geom_col() 
               + theme(axis_text_x = element_text(angle = 45 , hjust=1))
               + labs(title='Agency Program Codes', x='Program Codes', y='Number of Obligations', fill="Program Type")
        )
        return ax1

        

interactive(children=(Dropdown(description='place', options=('Access Services', 'Agoura Hills', 'Ala-Con Costa…

## Function for results by Prefix

In [6]:
@interact
def prefix_all_agencies_4(prefix_unique=df.prefix.sort_values().unique().tolist()):
    
    
    # graphs 
    prefixes = df[df.prefix== prefix_unique]
    
    prefix_count_num = (prefixes >> count(_.agency) >> arrange(-_.n))
    
    prefix_count = (prefixes >> count(_.agency) >> arrange(-_.n)).head(20)
    
    display(Markdown(f"**The number of agencies using {prefix_unique} is {len(prefix_count_num)}**"))
    
    # for the table- using one as some agencies only have one entry
    display(df[(df.prefix == prefix_unique)].sample(1))
    
    
    ax1 = (prefix_count
            >> ggplot(aes("agency", "n", fill="agency")) 
                + geom_col() 
                + theme(axis_text_x = element_text(angle = 45 , hjust=1))
                + labs(title='Top Agencies using Prefix', x='Agency', y='Number of Obligations', fill="Agency")
            )    
    return ax1
              

interactive(children=(Dropdown(description='prefix_unique', options=('6140000', 'ACNHPI', 'ACST-ER', 'ACSTP', …

## Function for Type of Work

In [7]:
top50 = (df >> count(_.type_of_work) >> arrange(-_.n)).head(50)

In [8]:
@interact
def findtypeofwork(work=top50.type_of_work.sort_values().unique().tolist()):
    
    display(Markdown(f"**Number of Obligations the top Agencies have in {work}**"))
    # graphs 
    worktypes = df[df.type_of_work== work]
    
    work_count = (worktypes >> count(_.agency) >> arrange(-_.n)).head(20)
    display(work_count)

    ax1 = (work_count
            >> ggplot(aes("agency", "n", fill="agency")) 
                + geom_col() 
                + theme(axis_text_x = element_text(angle = 45 , hjust=1))
                + labs(title='Agencies using Type of Work', x='Agency', y='Number of Obligations', fill="Agency")
            )    
    return ax1

interactive(children=(Dropdown(description='work', options=('Ac Overlay', 'Ac Overlay (tc)', 'Asphalt Concrete…

## Function to Filter Data

In [9]:
df_query = df[['prefix', 'agency', 'project_no','locode','dist','fed_requested', 'ac_requested', 'total_requested',
              'type_of_work','project_location', 'seq', 'mpo','prepared_y']]

In [10]:
#adding year
@interact
def get_query2(agency=df_query.agency.unique().tolist(),
             prefix=df_query.prefix.unique().tolist(),
             year=df_query.prepared_y.unique().tolist()):
   
    filtering = df_query[(df_query.agency==agency)&(df_query.prefix==prefix)&(df_query.prepared_y==year)]
    
    
    display(Markdown(f"**The number of obligations {agency} has in {prefix} during the year {year} is {len(filtering)}**"))
    
    display(filtering.head())
    
    

interactive(children=(Dropdown(description='agency', options=('Humboldt County', 'Mendocino County', 'Sacramen…